# 05 - Análisis Descriptivo: Clasificación de Eventos Delictivos

## Descripción

Este notebook genera **estadísticas descriptivas pre-calculadas** a partir del dataset de eventos delictivos de Santander. Los resultados se guardan en archivos JSON listos para ser consumidos por:

- **Tablero web**: KPIs, gráficos y filtros dinámicos
- **Chatbot**: Respuestas automáticas a preguntas frecuentes

## Dataset de Entrada

- **Archivo**: `data/gold/model/classification_event_dataset.parquet`
- **Granularidad**: Por evento delictivo individual
- **Contenido**: ~344k eventos con información temporal, demográfica y geoespacial

## Archivos de Salida

Los resultados se guardan en `models/descriptivo/classification_event/`:

| Archivo | Descripción | Uso Principal |
|---------|-------------|---------------|
| `resumen_general.json` | KPIs principales (total, período, municipios) | Header del dashboard |
| `distribucion_delitos.json` | Conteo y % por tipo de delito | Gráficos pie/barras |
| `distribucion_perfiles.json` | Conteo y % por perfil (agresor/víctima) | Segmentación |
| `analisis_temporal.json` | Tendencias por año, mes, trimestre | Gráficos de línea |
| `analisis_demografico.json` | Por grupo etario y género | Análisis poblacional |
| `analisis_geografico.json` | Top municipios, delitos por zona | Mapas de calor |
| `cruces_delito_perfil.json` | Relación delito-perfil | Heatmaps |
| `top_combinaciones.json` | Combinaciones más frecuentes | Rankings |
| `respuestas_chatbot.json` | Respuestas pre-generadas | Chatbot |

## Preguntas que Responde

### Para el Tablero
- ¿Cuántos delitos hay en total?
- ¿Cuál es la tendencia anual?
- ¿Qué municipios tienen más delitos?
- ¿Cómo se distribuyen los delitos por tipo?

### Para el Chatbot
- "¿Cuál es el delito más común?"
- "¿Qué municipio tiene más hurtos?"
- "¿Qué grupo etario es más afectado?"
- "¿Hay más delitos en fin de semana?"


In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', 50)

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'classification_event_dataset.parquet'
OUTPUT_DIR = BASE_DIR / 'models' / 'descriptivo' / 'classification_event'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Datos: {DATA_PATH}")
print(f"📁 Salida: {OUTPUT_DIR}")

📁 Datos: /Users/byverbel/quick_projects/Datos-al-Ecosistema/data/gold/model/classification_event_dataset.parquet
📁 Salida: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/classification_event


In [2]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f"✅ Datos cargados: {df.shape[0]:,} eventos × {df.shape[1]} columnas")
print(f"📅 Período: {df['anio'].min()} - {df['anio'].max()}")
print(f"📍 Municipios: {df['codigo_municipio'].nunique()}")
df.head(3)

✅ Datos cargados: 371,119 eventos × 64 columnas
📅 Período: 2010 - 2025
📍 Municipios: 87


,departamento,municipio,edad_persona,armas_medios,cantidad,fecha,genero,anio,delito,codigo_municipio,codigo_departamento,mes,dia,es_dia_semana,es_fin_de_semana,es_fin_mes,es_festivo,nombre_festivo,es_dia_laboral,origen,codigo_departamento_ctx,municipio_ctx,area,departamento_ctx,Shape_Leng,...,femenino_menores,masculino_adolescentes,masculino_adultos,masculino_menores,poblacion_total,poblacion_menores,poblacion_adultos,poblacion_adolescentes,area_km2,densidad_poblacional,centros_por_km2,proporcion_menores,proporcion_adultos,proporcion_adolescentes,fecha_ctx,trimestre,anio_mes,es_fin_ano,n_dias_semana,n_fines_de_semana,n_festivos,n_dias_laborales,mes_sin,mes_cos,perfil
0,SANTANDER,FLORIÁN,ADULTOS,NO REPORTADO,1.0,2010-01-05,MASCULINO,2010,ABIGEATO,68271,68,1,5,1,0,0,0,None,1,SCRAPING,68,FLORIÁN,176.599655,SANTANDER,0.892371,...,982,372,5335,1079,12682,2061,9959,662,176.599655,71.812145,0.022650,0.162514,0.785286,0.052200,2010-01-01,1,2010-01,0,3,3,2,1,0.5,0.866025,MASCULINO_ADULTOS
1,SANTANDER,PUENTE NACIONAL,ADULTOS,NO REPORTADO,1.0,2010-01-06,MASCULINO,2010,ABIGEATO,68572,68,1,6,1,0,0,0,None,1,SCRAPING,68,PUENTE NACIONAL,251.678548,SANTANDER,0.755909,...,1722,642,10725,1833,26902,3555,22030,1317,251.678548,106.890318,0.027813,0.132146,0.818898,0.048955,2010-01-01,1,2010-01,0,18,6,0,18,0.5,0.866025,MASCULINO_ADULTOS
2,SANTANDER,PUENTE NACIONAL,ADULTOS,ARMA DE FUEGO,1.0,2010-01-07,MASCULINO,2010,ABIGEATO,68572,68,1,7,1,0,0,0,None,1,SCRAPING,68,PUENTE NACIONAL,251.678548,SANTANDER,0.755909,...,1722,642,10725,1833,26902,3555,22030,1317,251.678548,106.890318,0.027813,0.132146,0.818898,0.048955,2010-01-01,1,2010-01,0,18,6,0,18,0.5,0.866025,MASCULINO_ADULTOS


---
## 1. Resumen General (KPIs)

In [3]:
# 1.1 Calcular KPIs principales
print("=" * 60)
print("RESUMEN GENERAL")
print("=" * 60)

resumen_general = {
    'generado': datetime.now().isoformat(),
    'dataset': 'classification_event_dataset.parquet',
    'total_eventos': int(len(df)),
    'periodo': {
        'anio_inicio': int(df['anio'].min()),
        'anio_fin': int(df['anio'].max()),
        'n_anios': int(df['anio'].nunique())
    },
    'geografia': {
        'n_municipios': int(df['codigo_municipio'].nunique()),
        'departamento': df['departamento'].iloc[0] if 'departamento' in df.columns else 'Santander'
    },
    'tipos_delito': {
        'n_tipos': int(df['delito'].nunique()),
        'tipos': df['delito'].unique().tolist()
    },
    'perfiles': {
        'n_perfiles': int(df['perfil'].nunique()),
        'perfiles': df['perfil'].unique().tolist()
    },
    'grupos_etarios': {
        'n_grupos': int(df['edad_persona'].nunique()),
        'grupos': df['edad_persona'].unique().tolist()
    },
    'promedios': {
        'eventos_por_anio': int(len(df) / df['anio'].nunique()),
        'eventos_por_mes': int(len(df) / (df['anio'].nunique() * 12)),
        'eventos_por_municipio': int(len(df) / df['codigo_municipio'].nunique())
    }
}

# Guardar
with open(OUTPUT_DIR / 'resumen_general.json', 'w', encoding='utf-8') as f:
    json.dump(resumen_general, f, indent=2, ensure_ascii=False)

print(f"\n📊 Total eventos: {resumen_general['total_eventos']:,}")
print(f"📅 Período: {resumen_general['periodo']['anio_inicio']} - {resumen_general['periodo']['anio_fin']}")
print(f"📍 Municipios: {resumen_general['geografia']['n_municipios']}")
print(f"🔹 Tipos de delito: {resumen_general['tipos_delito']['n_tipos']}")
print(f"🔹 Perfiles: {resumen_general['perfiles']['n_perfiles']}")
print(f"\n✅ Guardado: resumen_general.json")

RESUMEN GENERAL

📊 Total eventos: 371,119
📅 Período: 2010 - 2025
📍 Municipios: 87
🔹 Tipos de delito: 8
🔹 Perfiles: 12

✅ Guardado: resumen_general.json


---
## 2. Distribución de Delitos

In [4]:
# 2.1 Distribución por tipo de delito
print("=" * 60)
print("DISTRIBUCIÓN DE DELITOS")
print("=" * 60)

delitos_counts = df['delito'].value_counts()
delitos_pct = (delitos_counts / len(df) * 100).round(2)

distribucion_delitos = {
    'generado': datetime.now().isoformat(),
    'total': int(len(df)),
    'n_tipos': int(df['delito'].nunique()),
    'delito_mas_comun': {
        'nombre': delitos_counts.index[0],
        'cantidad': int(delitos_counts.iloc[0]),
        'porcentaje': float(delitos_pct.iloc[0])
    },
    'delito_menos_comun': {
        'nombre': delitos_counts.index[-1],
        'cantidad': int(delitos_counts.iloc[-1]),
        'porcentaje': float(delitos_pct.iloc[-1])
    },
    'distribucion': [
        {
            'delito': delito,
            'cantidad': int(count),
            'porcentaje': float(delitos_pct[delito])
        }
        for delito, count in delitos_counts.items()
    ],
    'ratio_desbalance': round(float(delitos_counts.max() / delitos_counts.min()), 1)
}

# Guardar
with open(OUTPUT_DIR / 'distribucion_delitos.json', 'w', encoding='utf-8') as f:
    json.dump(distribucion_delitos, f, indent=2, ensure_ascii=False)

print(f"\n📊 Distribución:")
for item in distribucion_delitos['distribucion']:
    print(f"   {item['delito']}: {item['cantidad']:,} ({item['porcentaje']}%)")

print(f"\n🏆 Más común: {distribucion_delitos['delito_mas_comun']['nombre']}")
print(f"📉 Menos común: {distribucion_delitos['delito_menos_comun']['nombre']}")
print(f"\n✅ Guardado: distribucion_delitos.json")

DISTRIBUCIÓN DE DELITOS

📊 Distribución:
   LESIONES: 123,600 (33.3%)
   HURTOS: 115,440 (31.11%)
   VIOLENCIA INTRAFAMILIAR: 65,327 (17.6%)
   AMENAZAS: 30,127 (8.12%)
   DELITOS SEXUALES: 23,087 (6.22%)
   HOMICIDIOS: 9,520 (2.57%)
   EXTORSION: 2,691 (0.73%)
   ABIGEATO: 1,327 (0.36%)

🏆 Más común: LESIONES
📉 Menos común: ABIGEATO

✅ Guardado: distribucion_delitos.json


---
## 3. Distribución de Perfiles

In [5]:
# 3.1 Distribución por perfil
print("=" * 60)
print("DISTRIBUCIÓN DE PERFILES")
print("=" * 60)

perfiles_counts = df['perfil'].value_counts()
perfiles_pct = (perfiles_counts / len(df) * 100).round(2)

distribucion_perfiles = {
    'generado': datetime.now().isoformat(),
    'total': int(len(df)),
    'n_perfiles': int(df['perfil'].nunique()),
    'perfil_mas_comun': {
        'nombre': perfiles_counts.index[0],
        'cantidad': int(perfiles_counts.iloc[0]),
        'porcentaje': float(perfiles_pct.iloc[0])
    },
    'distribucion': [
        {
            'perfil': perfil,
            'cantidad': int(count),
            'porcentaje': float(perfiles_pct[perfil])
        }
        for perfil, count in perfiles_counts.items()
    ]
}

# Guardar
with open(OUTPUT_DIR / 'distribucion_perfiles.json', 'w', encoding='utf-8') as f:
    json.dump(distribucion_perfiles, f, indent=2, ensure_ascii=False)

print(f"\n📊 Distribución:")
for item in distribucion_perfiles['distribucion']:
    print(f"   {item['perfil']}: {item['cantidad']:,} ({item['porcentaje']}%)")

print(f"\n✅ Guardado: distribucion_perfiles.json")

DISTRIBUCIÓN DE PERFILES

📊 Distribución:
   MASCULINO_ADULTOS: 169,708 (45.73%)
   FEMENINO_ADULTOS: 154,228 (41.56%)
   FEMENINO_ADOLESCENTES: 14,022 (3.78%)
   MASCULINO_ADOLESCENTES: 11,899 (3.21%)
   FEMENINO_MENORES: 10,710 (2.89%)
   MASCULINO_MENORES: 6,112 (1.65%)
   NO REPORTADO_nan: 2,520 (0.68%)
   FEMENINO_nan: 1,486 (0.4%)
   MASCULINO_nan: 246 (0.07%)
   NO REPORTA_ADULTOS: 153 (0.04%)
   NO REPORTA_MENORES: 20 (0.01%)
   NO REPORTA_ADOLESCENTES: 15 (0.0%)

✅ Guardado: distribucion_perfiles.json


---
## 4. Análisis Temporal

In [6]:
# 4.1 Análisis por año, mes, trimestre
print("=" * 60)
print("ANÁLISIS TEMPORAL")
print("=" * 60)

# Por año
por_anio = df.groupby('anio').size().to_dict()
por_anio = {int(k): int(v) for k, v in por_anio.items()}

# Por mes (agregado)
por_mes = df.groupby('mes').size().to_dict()
por_mes = {int(k): int(v) for k, v in por_mes.items()}
mes_max = max(por_mes, key=por_mes.get)
mes_min = min(por_mes, key=por_mes.get)

# Por trimestre
por_trimestre = df.groupby('trimestre').size().to_dict()
por_trimestre = {int(k): int(v) for k, v in por_trimestre.items()}

# Por día de semana
por_dia_semana = df.groupby('es_dia_semana').size().to_dict()
por_dia_semana = {int(k): int(v) for k, v in por_dia_semana.items()}

# Fin de semana vs laboral
fin_semana = int(df[df['es_fin_de_semana'] == 1].shape[0])
dia_laboral = int(df[df['es_fin_de_semana'] == 0].shape[0])

# Festivos
festivos = int(df[df['es_festivo'] == 1].shape[0])
no_festivos = int(df[df['es_festivo'] == 0].shape[0])

# Tendencia anual (variación año a año)
anios = sorted(por_anio.keys())
variacion_anual = []
for i in range(1, len(anios)):
    anio_actual = anios[i]
    anio_anterior = anios[i-1]
    var = ((por_anio[anio_actual] - por_anio[anio_anterior]) / por_anio[anio_anterior] * 100)
    variacion_anual.append({
        'anio': anio_actual,
        'variacion_pct': round(var, 2)
    })

# Delitos por año y tipo
delitos_por_anio = df.groupby(['anio', 'delito']).size().unstack(fill_value=0)
delitos_por_anio_dict = {}
for anio in delitos_por_anio.index:
    delitos_por_anio_dict[int(anio)] = {col: int(delitos_por_anio.loc[anio, col]) for col in delitos_por_anio.columns}

analisis_temporal = {
    'generado': datetime.now().isoformat(),
    'por_anio': por_anio,
    'por_mes': {
        'distribucion': por_mes,
        'mes_con_mas_delitos': mes_max,
        'mes_con_menos_delitos': mes_min
    },
    'por_trimestre': por_trimestre,
    'por_tipo_dia': {
        'fin_de_semana': fin_semana,
        'dia_laboral': dia_laboral,
        'pct_fin_semana': round(fin_semana / len(df) * 100, 2)
    },
    'festivos': {
        'en_festivo': festivos,
        'no_festivo': no_festivos,
        'pct_festivo': round(festivos / len(df) * 100, 2)
    },
    'variacion_anual': variacion_anual,
    'delitos_por_anio': delitos_por_anio_dict
}

# Guardar
with open(OUTPUT_DIR / 'analisis_temporal.json', 'w', encoding='utf-8') as f:
    json.dump(analisis_temporal, f, indent=2, ensure_ascii=False)

print(f"\n📅 Por año:")
for anio, count in por_anio.items():
    print(f"   {anio}: {count:,}")

print(f"\n📆 Mes con más delitos: {mes_max} ({por_mes[mes_max]:,})")
print(f"📆 Mes con menos delitos: {mes_min} ({por_mes[mes_min]:,})")

print(f"\n🗓️ Fin de semana: {fin_semana:,} ({analisis_temporal['por_tipo_dia']['pct_fin_semana']}%)")
print(f"🗓️ Día laboral: {dia_laboral:,}")

print(f"\n📈 Variación anual:")
for v in variacion_anual:
    signo = '+' if v['variacion_pct'] > 0 else ''
    print(f"   {v['anio']}: {signo}{v['variacion_pct']}%")

print(f"\n✅ Guardado: analisis_temporal.json")

ANÁLISIS TEMPORAL

📅 Por año:
   2010: 17,743
   2011: 17,628
   2012: 19,153
   2013: 19,447
   2014: 20,309
   2015: 21,045
   2016: 21,290
   2017: 22,269
   2018: 35,016
   2019: 33,061
   2020: 20,540
   2021: 23,744
   2022: 16,215
   2023: 23,033
   2024: 21,378
   2025: 39,248

📆 Mes con más delitos: 3 (33,044)
📆 Mes con menos delitos: 12 (26,421)

🗓️ Fin de semana: 120,185 (32.38%)
🗓️ Día laboral: 250,934

📈 Variación anual:
   2011: -0.65%
   2012: +8.65%
   2013: +1.54%
   2014: +4.43%
   2015: +3.62%
   2016: +1.16%
   2017: +4.6%
   2018: +57.24%
   2019: -5.58%
   2020: -37.87%
   2021: +15.6%
   2022: -31.71%
   2023: +42.05%
   2024: -7.19%
   2025: +83.59%

✅ Guardado: analisis_temporal.json


---
## 5. Análisis Demográfico

In [7]:
# 5.1 Análisis por grupo etario y género
print("=" * 60)
print("ANÁLISIS DEMOGRÁFICO")
print("=" * 60)

# Por grupo etario
por_edad = df['edad_persona'].value_counts().to_dict()
por_edad = {str(k): int(v) for k, v in por_edad.items()}
por_edad_pct = {k: round(v / len(df) * 100, 2) for k, v in por_edad.items()}

# Por género
por_genero = df['genero'].value_counts().to_dict()
por_genero = {str(k): int(v) for k, v in por_genero.items()}
por_genero_pct = {k: round(v / len(df) * 100, 2) for k, v in por_genero.items()}

# Delitos por grupo etario
delitos_por_edad = df.groupby(['edad_persona', 'delito']).size().unstack(fill_value=0)
delitos_por_edad_dict = {}
for edad in delitos_por_edad.index:
    delitos_por_edad_dict[str(edad)] = {col: int(delitos_por_edad.loc[edad, col]) for col in delitos_por_edad.columns}

# Delitos por género
delitos_por_genero = df.groupby(['genero', 'delito']).size().unstack(fill_value=0)
delitos_por_genero_dict = {}
for genero in delitos_por_genero.index:
    delitos_por_genero_dict[str(genero)] = {col: int(delitos_por_genero.loc[genero, col]) for col in delitos_por_genero.columns}

# Grupo etario más afectado por delito
edad_mas_afectada_por_delito = {}
for delito in df['delito'].unique():
    df_delito = df[df['delito'] == delito]
    edad_top = df_delito['edad_persona'].value_counts().index[0]
    edad_mas_afectada_por_delito[delito] = str(edad_top)

analisis_demografico = {
    'generado': datetime.now().isoformat(),
    'por_grupo_etario': {
        'distribucion': por_edad,
        'porcentajes': por_edad_pct,
        'grupo_mas_frecuente': max(por_edad, key=por_edad.get)
    },
    'por_genero': {
        'distribucion': por_genero,
        'porcentajes': por_genero_pct
    },
    'delitos_por_grupo_etario': delitos_por_edad_dict,
    'delitos_por_genero': delitos_por_genero_dict,
    'grupo_etario_mas_afectado_por_delito': edad_mas_afectada_por_delito
}

# Guardar
with open(OUTPUT_DIR / 'analisis_demografico.json', 'w', encoding='utf-8') as f:
    json.dump(analisis_demografico, f, indent=2, ensure_ascii=False)

print(f"\n👥 Por grupo etario:")
for edad, count in por_edad.items():
    print(f"   {edad}: {count:,} ({por_edad_pct[edad]}%)")

print(f"\n👤 Por género:")
for genero, count in por_genero.items():
    print(f"   {genero}: {count:,} ({por_genero_pct[genero]}%)")

print(f"\n🎯 Grupo etario más afectado por delito:")
for delito, edad in edad_mas_afectada_por_delito.items():
    print(f"   {delito}: {edad}")

print(f"\n✅ Guardado: analisis_demografico.json")

ANÁLISIS DEMOGRÁFICO

👥 Por grupo etario:
   ADULTOS: 324,089 (87.33%)
   ADOLESCENTES: 25,936 (6.99%)
   MENORES: 16,842 (4.54%)

👤 Por género:
   MASCULINO: 187,965 (50.65%)
   FEMENINO: 180,446 (48.62%)
   NO REPORTADO: 2,520 (0.68%)
   NO REPORTA: 188 (0.05%)

🎯 Grupo etario más afectado por delito:
   ABIGEATO: ADULTOS
   AMENAZAS: ADULTOS
   DELITOS SEXUALES: MENORES
   EXTORSION: ADULTOS
   HOMICIDIOS: ADULTOS
   HURTOS: ADULTOS
   LESIONES: ADULTOS
   VIOLENCIA INTRAFAMILIAR: ADULTOS

✅ Guardado: analisis_demografico.json


---
## 6. Análisis Geográfico

In [8]:
# 6.1 Análisis por municipio
print("=" * 60)
print("ANÁLISIS GEOGRÁFICO")
print("=" * 60)

# Por municipio (código)
por_municipio = df.groupby('codigo_municipio').size().sort_values(ascending=False)

# Obtener nombre del municipio si existe
if 'municipio' in df.columns:
    muni_nombres = df.groupby('codigo_municipio')['municipio'].first().to_dict()
else:
    muni_nombres = {}

# Top 10 municipios
top_10_municipios = []
for codigo, count in por_municipio.head(10).items():
    nombre = muni_nombres.get(codigo, f"Municipio {codigo}")
    top_10_municipios.append({
        'codigo': int(codigo),
        'nombre': nombre,
        'cantidad': int(count),
        'porcentaje': round(count / len(df) * 100, 2)
    })

# Delitos por municipio (top 10)
delitos_por_municipio = {}
for codigo in por_municipio.head(10).index:
    df_muni = df[df['codigo_municipio'] == codigo]
    delitos_count = df_muni['delito'].value_counts().to_dict()
    nombre = muni_nombres.get(codigo, f"Municipio {codigo}")
    delitos_por_municipio[nombre] = {str(k): int(v) for k, v in delitos_count.items()}

# Municipio con más de cada delito
municipio_top_por_delito = {}
for delito in df['delito'].unique():
    df_delito = df[df['delito'] == delito]
    muni_top = df_delito['codigo_municipio'].value_counts().index[0]
    nombre = muni_nombres.get(muni_top, f"Municipio {muni_top}")
    count = int(df_delito[df_delito['codigo_municipio'] == muni_top].shape[0])
    municipio_top_por_delito[delito] = {
        'codigo': int(muni_top),
        'nombre': nombre,
        'cantidad': count
    }

# Estadísticas de densidad
densidad_stats = {
    'promedio': round(float(df['densidad_poblacional'].mean()), 2),
    'mediana': round(float(df['densidad_poblacional'].median()), 2),
    'max': round(float(df['densidad_poblacional'].max()), 2),
    'min': round(float(df['densidad_poblacional'].min()), 2)
}

analisis_geografico = {
    'generado': datetime.now().isoformat(),
    'n_municipios': int(df['codigo_municipio'].nunique()),
    'top_10_municipios': top_10_municipios,
    'delitos_por_municipio_top10': delitos_por_municipio,
    'municipio_con_mas_de_cada_delito': municipio_top_por_delito,
    'densidad_poblacional': densidad_stats
}

# Guardar
with open(OUTPUT_DIR / 'analisis_geografico.json', 'w', encoding='utf-8') as f:
    json.dump(analisis_geografico, f, indent=2, ensure_ascii=False)

print(f"\n📍 Top 10 municipios con más delitos:")
for i, muni in enumerate(top_10_municipios, 1):
    print(f"   {i}. {muni['nombre']}: {muni['cantidad']:,} ({muni['porcentaje']}%)")

print(f"\n🏆 Municipio con más de cada delito:")
for delito, info in municipio_top_por_delito.items():
    print(f"   {delito}: {info['nombre']} ({info['cantidad']:,})")

print(f"\n✅ Guardado: analisis_geografico.json")

ANÁLISIS GEOGRÁFICO

📍 Top 10 municipios con más delitos:
   1. BUCARAMANGA (CT): 114,940 (30.97%)
   2. BARRANCABERMEJA: 48,660 (13.11%)
   3. FLORIDABLANCA: 43,912 (11.83%)
   4. GIRÓN: 33,572 (9.05%)
   5. PIEDECUESTA: 28,973 (7.81%)
   6. SAN GIL: 10,767 (2.9%)
   7. BARBOSA: 8,474 (2.28%)
   8. SABANA DE TORRES: 6,777 (1.83%)
   9. CIMITARRA: 6,338 (1.71%)
   10. LEBRIJA: 6,289 (1.69%)

🏆 Municipio con más de cada delito:
   ABIGEATO: BARRANCABERMEJA (283)
   AMENAZAS: BUCARAMANGA (CT) (7,376)
   DELITOS SEXUALES: BUCARAMANGA (CT) (6,409)
   EXTORSION: BUCARAMANGA (CT) (901)
   HOMICIDIOS: BUCARAMANGA (CT) (2,599)
   HURTOS: BUCARAMANGA (CT) (44,381)
   LESIONES: BUCARAMANGA (CT) (36,090)
   VIOLENCIA INTRAFAMILIAR: BUCARAMANGA (CT) (17,170)

✅ Guardado: analisis_geografico.json


---
## 7. Cruces Delito-Perfil

In [9]:
# 7.1 Relación entre delito y perfil
print("=" * 60)
print("CRUCES DELITO-PERFIL")
print("=" * 60)

# Crosstab absoluto
cross_abs = pd.crosstab(df['delito'], df['perfil'])

# Crosstab porcentual (por delito)
cross_pct = pd.crosstab(df['delito'], df['perfil'], normalize='index') * 100

# Convertir a diccionarios
cruce_absoluto = {}
cruce_porcentual = {}
for delito in cross_abs.index:
    cruce_absoluto[delito] = {perfil: int(cross_abs.loc[delito, perfil]) for perfil in cross_abs.columns}
    cruce_porcentual[delito] = {perfil: round(float(cross_pct.loc[delito, perfil]), 2) for perfil in cross_pct.columns}

# Perfil dominante por delito
perfil_dominante = {}
for delito in cross_pct.index:
    perfil_top = cross_pct.loc[delito].idxmax()
    pct = round(float(cross_pct.loc[delito, perfil_top]), 2)
    perfil_dominante[delito] = {
        'perfil': perfil_top,
        'porcentaje': pct
    }

cruces_delito_perfil = {
    'generado': datetime.now().isoformat(),
    'cruce_absoluto': cruce_absoluto,
    'cruce_porcentual': cruce_porcentual,
    'perfil_dominante_por_delito': perfil_dominante
}

# Guardar
with open(OUTPUT_DIR / 'cruces_delito_perfil.json', 'w', encoding='utf-8') as f:
    json.dump(cruces_delito_perfil, f, indent=2, ensure_ascii=False)

print(f"\n📊 Perfil dominante por delito:")
for delito, info in perfil_dominante.items():
    print(f"   {delito}: {info['perfil']} ({info['porcentaje']}%)")

print(f"\n✅ Guardado: cruces_delito_perfil.json")

CRUCES DELITO-PERFIL

📊 Perfil dominante por delito:
   ABIGEATO: MASCULINO_ADULTOS (80.33%)
   AMENAZAS: MASCULINO_ADULTOS (55.38%)
   DELITOS SEXUALES: FEMENINO_MENORES (31.88%)
   EXTORSION: MASCULINO_ADULTOS (67.56%)
   HOMICIDIOS: MASCULINO_ADULTOS (79.46%)
   HURTOS: MASCULINO_ADULTOS (52.66%)
   LESIONES: MASCULINO_ADULTOS (54.89%)
   VIOLENCIA INTRAFAMILIAR: FEMENINO_ADULTOS (70.66%)

✅ Guardado: cruces_delito_perfil.json


---
## 8. Top Combinaciones

In [10]:
# 8.1 Combinaciones más frecuentes
print("=" * 60)
print("TOP COMBINACIONES")
print("=" * 60)

# Combinación: delito + perfil + grupo etario
combo_counts = df.groupby(['delito', 'perfil', 'edad_persona']).size().reset_index(name='cantidad')
combo_counts = combo_counts.sort_values('cantidad', ascending=False)

top_combinaciones = []
for _, row in combo_counts.head(20).iterrows():
    top_combinaciones.append({
        'delito': row['delito'],
        'perfil': row['perfil'],
        'grupo_etario': str(row['edad_persona']),
        'cantidad': int(row['cantidad']),
        'porcentaje': round(row['cantidad'] / len(df) * 100, 2)
    })

# Combinación: municipio + delito (top 15)
combo_muni = df.groupby(['codigo_municipio', 'delito']).size().reset_index(name='cantidad')
combo_muni = combo_muni.sort_values('cantidad', ascending=False)

top_municipio_delito = []
for _, row in combo_muni.head(15).iterrows():
    nombre = muni_nombres.get(row['codigo_municipio'], f"Municipio {row['codigo_municipio']}")
    top_municipio_delito.append({
        'municipio': nombre,
        'codigo_municipio': int(row['codigo_municipio']),
        'delito': row['delito'],
        'cantidad': int(row['cantidad'])
    })

top_combinaciones_data = {
    'generado': datetime.now().isoformat(),
    'top_20_delito_perfil_edad': top_combinaciones,
    'top_15_municipio_delito': top_municipio_delito
}

# Guardar
with open(OUTPUT_DIR / 'top_combinaciones.json', 'w', encoding='utf-8') as f:
    json.dump(top_combinaciones_data, f, indent=2, ensure_ascii=False)

print(f"\n🔝 Top 10 combinaciones (delito + perfil + grupo etario):")
for i, combo in enumerate(top_combinaciones[:10], 1):
    print(f"   {i}. {combo['delito']} | {combo['perfil']} | {combo['grupo_etario']}: {combo['cantidad']:,}")

print(f"\n🔝 Top 10 combinaciones (municipio + delito):")
for i, combo in enumerate(top_municipio_delito[:10], 1):
    print(f"   {i}. {combo['municipio']} | {combo['delito']}: {combo['cantidad']:,}")

print(f"\n✅ Guardado: top_combinaciones.json")

TOP COMBINACIONES

🔝 Top 10 combinaciones (delito + perfil + grupo etario):
   1. LESIONES | MASCULINO_ADULTOS | ADULTOS: 67,848
   2. HURTOS | MASCULINO_ADULTOS | ADULTOS: 60,786
   3. HURTOS | FEMENINO_ADULTOS | ADULTOS: 48,276
   4. VIOLENCIA INTRAFAMILIAR | FEMENINO_ADULTOS | ADULTOS: 46,160
   5. LESIONES | FEMENINO_ADULTOS | ADULTOS: 40,323
   6. AMENAZAS | MASCULINO_ADULTOS | ADULTOS: 16,684
   7. VIOLENCIA INTRAFAMILIAR | MASCULINO_ADULTOS | ADULTOS: 13,192
   8. AMENAZAS | FEMENINO_ADULTOS | ADULTOS: 12,081
   9. HOMICIDIOS | MASCULINO_ADULTOS | ADULTOS: 7,565
   10. DELITOS SEXUALES | FEMENINO_MENORES | MENORES: 7,359

🔝 Top 10 combinaciones (municipio + delito):
   1. BUCARAMANGA (CT) | HURTOS: 44,381
   2. BUCARAMANGA (CT) | LESIONES: 36,090
   3. BUCARAMANGA (CT) | VIOLENCIA INTRAFAMILIAR: 17,170
   4. FLORIDABLANCA | HURTOS: 16,587
   5. BARRANCABERMEJA | HURTOS: 14,429
   6. BARRANCABERMEJA | LESIONES: 13,648
   7. FLORIDABLANCA | LESIONES: 13,341
   8. GIRÓN | LESIONES:

---
## 9. Respuestas Pre-generadas para Chatbot

In [11]:
# 9.1 Generar respuestas para preguntas frecuentes del chatbot
print("=" * 60)
print("RESPUESTAS PARA CHATBOT")
print("=" * 60)

# Obtener datos necesarios
delito_top = distribucion_delitos['delito_mas_comun']
perfil_top = distribucion_perfiles['perfil_mas_comun']
muni_top = top_10_municipios[0] if top_10_municipios else None
edad_top = analisis_demografico['por_grupo_etario']['grupo_mas_frecuente']
mes_mas_delitos = analisis_temporal['por_mes']['mes_con_mas_delitos']

meses_nombres = {
    1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril',
    5: 'mayo', 6: 'junio', 7: 'julio', 8: 'agosto',
    9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'
}

respuestas_chatbot = {
    'generado': datetime.now().isoformat(),
    'preguntas_respuestas': [
        {
            'pregunta': '¿Cuántos delitos hay registrados en total?',
            'respuesta': f"En el período {resumen_general['periodo']['anio_inicio']}-{resumen_general['periodo']['anio_fin']} se registraron {resumen_general['total_eventos']:,} eventos delictivos en Santander."
        },
        {
            'pregunta': '¿Cuál es el delito más común?',
            'respuesta': f"El delito más común es {delito_top['nombre']} con {delito_top['cantidad']:,} casos ({delito_top['porcentaje']}% del total)."
        },
        {
            'pregunta': '¿Cuál es el perfil más frecuente?',
            'respuesta': f"El perfil más frecuente es {perfil_top['nombre']} con {perfil_top['cantidad']:,} casos ({perfil_top['porcentaje']}% del total)."
        },
        {
            'pregunta': '¿Qué municipio tiene más delitos?',
            'respuesta': f"El municipio con más delitos es {muni_top['nombre']} con {muni_top['cantidad']:,} casos ({muni_top['porcentaje']}% del total)." if muni_top else "No hay datos de municipios."
        },
        {
            'pregunta': '¿Qué grupo etario es más afectado?',
            'respuesta': f"El grupo etario más afectado es {edad_top} con {por_edad[edad_top]:,} casos ({por_edad_pct[edad_top]}% del total)."
        },
        {
            'pregunta': '¿Cuál es el mes con más delitos?',
            'respuesta': f"El mes con más delitos es {meses_nombres[mes_mas_delitos]} con {por_mes[mes_mas_delitos]:,} casos."
        },
        {
            'pregunta': '¿Cuántos tipos de delitos hay?',
            'respuesta': f"Hay {resumen_general['tipos_delito']['n_tipos']} tipos de delitos: {', '.join(resumen_general['tipos_delito']['tipos'])}."
        },
        {
            'pregunta': '¿Hay más delitos en fin de semana o días laborales?',
            'respuesta': f"Hay más delitos en días laborales ({dia_laboral:,}) que en fines de semana ({fin_semana:,}). Los fines de semana representan el {analisis_temporal['por_tipo_dia']['pct_fin_semana']}% del total."
        },
        {
            'pregunta': '¿Cuántos municipios tienen registros de delitos?',
            'respuesta': f"Hay {resumen_general['geografia']['n_municipios']} municipios de Santander con registros de delitos."
        }
    ],
    'respuestas_por_delito': {},
    'respuestas_por_municipio': {}
}

# Generar respuestas por cada tipo de delito
for item in distribucion_delitos['distribucion']:
    delito = item['delito']
    perfil_dom = perfil_dominante.get(delito, {})
    edad_afectada = edad_mas_afectada_por_delito.get(delito, 'N/A')
    muni_info = municipio_top_por_delito.get(delito, {})
    
    respuestas_chatbot['respuestas_por_delito'][delito] = {
        'total': item['cantidad'],
        'porcentaje': item['porcentaje'],
        'perfil_principal': perfil_dom.get('perfil', 'N/A'),
        'grupo_etario_mas_afectado': edad_afectada,
        'municipio_con_mas_casos': muni_info.get('nombre', 'N/A'),
        'respuesta': f"{delito}: {item['cantidad']:,} casos ({item['porcentaje']}%). El perfil más común es {perfil_dom.get('perfil', 'N/A')} y el grupo etario más afectado es {edad_afectada}. El municipio con más casos es {muni_info.get('nombre', 'N/A')}."
    }

# Generar respuestas por municipio (top 10)
for muni in top_10_municipios:
    nombre = muni['nombre']
    delitos_muni = delitos_por_municipio.get(nombre, {})
    delito_top_muni = max(delitos_muni, key=delitos_muni.get) if delitos_muni else 'N/A'
    
    respuestas_chatbot['respuestas_por_municipio'][nombre] = {
        'total': muni['cantidad'],
        'porcentaje': muni['porcentaje'],
        'delito_mas_comun': delito_top_muni,
        'respuesta': f"{nombre}: {muni['cantidad']:,} delitos ({muni['porcentaje']}%). El delito más común es {delito_top_muni}."
    }

# Guardar
with open(OUTPUT_DIR / 'respuestas_chatbot.json', 'w', encoding='utf-8') as f:
    json.dump(respuestas_chatbot, f, indent=2, ensure_ascii=False)

print(f"\n💬 Preguntas pre-generadas:")
for qa in respuestas_chatbot['preguntas_respuestas']:
    print(f"\n❓ {qa['pregunta']}")
    print(f"✅ {qa['respuesta']}")

print(f"\n✅ Guardado: respuestas_chatbot.json")

RESPUESTAS PARA CHATBOT

💬 Preguntas pre-generadas:

❓ ¿Cuántos delitos hay registrados en total?
✅ En el período 2010-2025 se registraron 371,119 eventos delictivos en Santander.

❓ ¿Cuál es el delito más común?
✅ El delito más común es LESIONES con 123,600 casos (33.3% del total).

❓ ¿Cuál es el perfil más frecuente?
✅ El perfil más frecuente es MASCULINO_ADULTOS con 169,708 casos (45.73% del total).

❓ ¿Qué municipio tiene más delitos?
✅ El municipio con más delitos es BUCARAMANGA (CT) con 114,940 casos (30.97% del total).

❓ ¿Qué grupo etario es más afectado?
✅ El grupo etario más afectado es ADULTOS con 324,089 casos (87.33% del total).

❓ ¿Cuál es el mes con más delitos?
✅ El mes con más delitos es marzo con 33,044 casos.

❓ ¿Cuántos tipos de delitos hay?
✅ Hay 8 tipos de delitos: ABIGEATO, AMENAZAS, DELITOS SEXUALES, EXTORSION, HOMICIDIOS, HURTOS, LESIONES, VIOLENCIA INTRAFAMILIAR.

❓ ¿Hay más delitos en fin de semana o días laborales?
✅ Hay más delitos en días laborales (250,93

---
## 10. Resumen de Archivos Generados

In [12]:
# 10.1 Listar archivos generados
print("=" * 60)
print("ARCHIVOS GENERADOS")
print("=" * 60)

print(f"\n📁 Directorio: {OUTPUT_DIR}")
print(f"\n📋 Archivos:")

total_size = 0
for file in sorted(OUTPUT_DIR.iterdir()):
    size = file.stat().st_size / 1024
    total_size += size
    print(f"   - {file.name} ({size:.1f} KB)")

print(f"\n📊 Total: {total_size:.1f} KB")

print("\n" + "=" * 60)
print("✅ ANÁLISIS DESCRIPTIVO COMPLETADO")
print("=" * 60)
print(f"\n🎯 Uso en Tablero:")
print("   - Cargar JSONs para mostrar KPIs y gráficos")
print("   - Usar distribuciones para filtros dinámicos")
print("   - Mostrar top municipios y tendencias")

print(f"\n🤖 Uso en Chatbot:")
print("   - Usar respuestas_chatbot.json para respuestas rápidas")
print("   - Consultar respuestas_por_delito para preguntas específicas")
print("   - Consultar respuestas_por_municipio para datos geográficos")

ARCHIVOS GENERADOS

📁 Directorio: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/classification_event

📋 Archivos:
   - analisis_demografico.json (2.5 KB)
   - analisis_geografico.json (4.6 KB)
   - analisis_temporal.json (5.6 KB)
   - cruces_delito_perfil.json (7.4 KB)
   - distribucion_delitos.json (1.1 KB)
   - distribucion_perfiles.json (1.4 KB)
   - respuestas_chatbot.json (7.0 KB)
   - resumen_general.json (1.1 KB)
   - top_combinaciones.json (5.4 KB)

📊 Total: 36.0 KB

✅ ANÁLISIS DESCRIPTIVO COMPLETADO

🎯 Uso en Tablero:
   - Cargar JSONs para mostrar KPIs y gráficos
   - Usar distribuciones para filtros dinámicos
   - Mostrar top municipios y tendencias

🤖 Uso en Chatbot:
   - Usar respuestas_chatbot.json para respuestas rápidas
   - Consultar respuestas_por_delito para preguntas específicas
   - Consultar respuestas_por_municipio para datos geográficos


---
## 11. Ejemplo de Uso de los Datos

### Para el Tablero Web

```python
import json

# Cargar datos
with open('models/descriptivo/classification_event/resumen_general.json') as f:
    resumen = json.load(f)

# Mostrar KPIs
print(f"Total eventos: {resumen['total_eventos']:,}")
print(f"Período: {resumen['periodo']['anio_inicio']}-{resumen['periodo']['anio_fin']}")
```

### Para el Chatbot

```python
import json

# Cargar respuestas
with open('models/descriptivo/classification_event/respuestas_chatbot.json') as f:
    respuestas = json.load(f)

# Buscar respuesta por delito
delito = 'HURTOS'
info = respuestas['respuestas_por_delito'][delito]
print(info['respuesta'])

# Buscar respuesta por municipio
municipio = 'Bucaramanga'
info = respuestas['respuestas_por_municipio'][municipio]
print(info['respuesta'])
```